# Coursework 2
(core codebase to be imported as modules)

In [1]:
import numpy as np
from perceptron import Perceptron, KernelPerceptron
from kernels import polynomial_kernel, radial_basis_kernel

In [2]:
from utilities import preprocessing, train_test_split,y_encode,data_split

In [3]:
data = np.loadtxt("zipcombo.dat")

This will be a one vs all scenario.

In [18]:
# first class is zero

# matrix to hold predictions


X,y = data_split(data,y_col=0)
X_train,X_test,y_train,y_test = train_test_split(X,y)

predictions = np.zeros((X_test.shape[0],10))


y_train_i = y_encode(y_train,0)

P = Perceptron() # quadratic kernel
P.train(X_train,y_train_i)
print(P.M)
predictions[:,0] = P.predict_proba(X_test)

[209.]


In [19]:
for i in range(1,10):
    y_train_i = y_encode(y_train,i)
    print(y_train_i[0:10])
    P = Perceptron() # quadratic kernel
    print(P.M)
    P.train(X_train,y_train_i)
    print(P.M)
    predictions[:,i] = P.predict_proba(X_test)

[-1  1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[97.]
[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1]
0
[244.]
[-1 -1 -1 -1 -1 -1 -1  1 -1 -1]
0
[286.]
[-1 -1 -1  1  1 -1 -1 -1 -1 -1]
0
[301.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[302.]
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1]
0
[187.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[214.]
[-1 -1  1 -1 -1 -1 -1 -1 -1 -1]
0
[314.]
[-1 -1 -1 -1 -1 -1 -1 -1  1 -1]
0
[321.]


In [20]:
preds = np.argmax(predictions,axis=1)

In [9]:
preds.shape

(3100,)

In [21]:
preds[1:20]

array([7, 0, 1, 5, 1, 6, 6, 0, 9, 6, 0, 9, 0, 6, 0, 2, 4, 4, 0],
      dtype=int64)

In [22]:
y_test[1:20]

array([7., 0., 1., 5., 1., 6., 6., 0., 7., 6., 0., 9., 8., 6., 0., 2., 4.,
       4., 0.])